[Consumer Complaint Database](https://www.consumerfinance.gov/data-research/consumer-complaints/#download-the-data)

In [5]:
!wget https://data.consumerfinance.gov/api/views/s6ew-h6mp/rows.csv?accessType=DOWNLOAD

--2019-05-25 15:43:53--  https://data.consumerfinance.gov/api/views/s6ew-h6mp/rows.csv?accessType=DOWNLOAD
Resolving data.consumerfinance.gov (data.consumerfinance.gov)... 52.206.68.26, 52.206.140.205, 52.206.140.199
Connecting to data.consumerfinance.gov (data.consumerfinance.gov)|52.206.68.26|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/csv]
Saving to: ‘rows.csv?accessType=DOWNLOAD’

rows.csv?accessType     [       <=>          ] 706.62M  5.37MB/s    in 79s     

2019-05-25 15:45:15 (8.91 MB/s) - ‘rows.csv?accessType=DOWNLOAD’ saved [740947778]



In [7]:
!wc -l rows.csv\?accessType\=DOWNLOAD

1982835 rows.csv?accessType=DOWNLOAD


In [0]:
!mv rows.csv\?accessType\=DOWNLOAD consumer_complaints.csv

In [12]:
!head -5 consumer_complaints.csv

Date received,Product,Sub-product,Issue,Sub-issue,Consumer complaint narrative,Company public response,Company,State,ZIP code,Tags,Consumer consent provided?,Submitted via,Date sent to company,Company response to consumer,Timely response?,Consumer disputed?,Complaint ID
05/24/2019,Debt collection,Medical debt,Written notification about debt,Didn't receive enough information to verify debt,,Company believes it acted appropriately as authorized by contract or law,"GLA Collection Company, Inc.",KY,40220,,Consent not provided,Web,05/24/2019,Closed with explanation,Yes,N/A,3252999
05/24/2019,Credit card or prepaid card,General-purpose credit card or charge card,Problem with a purchase shown on your statement,Credit card company isn't resolving a dispute about a purchase on your statement,,,NAVY FEDERAL CREDIT UNION,VA,22153,,,Web,05/24/2019,In progress,Yes,N/A,3252728
05/24/2019,Debt collection,Medical debt,Written notification about debt,Didn't receive notice of right to dispute,,Company b

In [14]:
!head -1 consumer_complaints.csv

Date received,Product,Sub-product,Issue,Sub-issue,Consumer complaint narrative,Company public response,Company,State,ZIP code,Tags,Consumer consent provided?,Submitted via,Date sent to company,Company response to consumer,Timely response?,Consumer disputed?,Complaint ID


In [0]:
columns = """Date received,Product,Sub-product,Issue,Sub-issue,Consumer complaint narrative,Company public response,Company,State,
ZIP code,Tags,Consumer consent provided?,Submitted via,Date sent to company,Company response to consumer,Timely response?,
Consumer disputed?,Complaint ID"""

In [0]:
columns = columns.replace(" ",'_').replace('\n','').replace('?','').lower().split(",")
# converting column names to lower case and replaced space by underscore

In [18]:
columns

['date_received',
 'product',
 'sub-product',
 'issue',
 'sub-issue',
 'consumer_complaint_narrative',
 'company_public_response',
 'company',
 'state',
 'zip_code',
 'tags',
 'consumer_consent_provided',
 'submitted_via',
 'date_sent_to_company',
 'company_response_to_consumer',
 'timely_response',
 'consumer_disputed',
 'complaint_id']

In [19]:
len(columns)

18

In [20]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www-eu.apache.org/dist/spark/spark-2.3.3/spark-2.3.3-bin-hadoop2.7.tgz
!tar xf spark-2.3.3-bin-hadoop2.7.tgz
!pip install -q findspark
!pip install pyspark

     |████████████████████████████████| 215.6MB 104kB/s 
     |████████████████████████████████| 204kB 39.3MB/s 
  Stored in directory: /root/.cache/pip/wheels/8d/20/f0/b30e2024226dc112e256930dd2cd4f06d00ab053c86278dcf3
Successfully built pyspark


In [0]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.3.3-bin-hadoop2.7"

**Initializing SparkSession**

Using stand alone Spark and instantiated SparkSession with Hive support which creates spark-warehouse.

In [0]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.enableHiveSupport().config("spark.network.timeout", '200s').getOrCreate()
from pyspark.sql.types import *

In [0]:
# Define schema
consumer_schema = StructType([StructField(colName, StringType(), True) for colName in columns]) 

In [0]:
# Read Data
cc = spark.read.csv("consumer_complaints.csv",
                         header = True, 
                         schema = consumer_schema)

In [29]:
cc.show(10, truncate = False)

+-------------+----------------------------------------------------------------------------+------------------------------------------+-----------------------------------------------+--------------------------------------------------------------------------------+----------------------------+------------------------------------------------------------------------+---------------------------------+-----+--------+-------------+-------------------------+-------------+--------------------+----------------------------+---------------+-----------------+------------+
|date_received|product                                                                     |sub-product                               |issue                                          |sub-issue                                                                       |consumer_complaint_narrative|company_public_response                                                 |company                          |state|zip_code|tags         |cons

In [30]:
!tail -10 consumer_complaints.csv

12/01/2011,Credit card,,Credit determination,,,,TD BANK US HOLDING COMPANY,IL,60302,,N/A,Referral,12/01/2011,Closed without relief,Yes,No,2207
12/01/2011,Mortgage,Home equity loan or line of credit,"Loan modification,collection,foreclosure",,,,"BANK OF AMERICA, NATIONAL ASSOCIATION",MI,48197,,N/A,Phone,12/02/2011,Closed without relief,No,No,2231
12/01/2011,Mortgage,Conventional adjustable mortgage (ARM),"Loan servicing, payments, escrow account",,,,"BANK OF AMERICA, NATIONAL ASSOCIATION",CO,80920,,N/A,Web,12/01/2011,Closed with relief,Yes,No,2315
12/01/2011,Credit card,,Other,,,,HSBC NORTH AMERICA HOLDINGS INC.,MD,20747,,N/A,Referral,12/19/2011,Closed without relief,Yes,No,2254
12/01/2011,Credit card,,Credit determination,,,,"CITIBANK, N.A.",MO,63301,,N/A,Web,12/02/2011,Closed with relief,Yes,No,2396
12/01/2011,Credit card,,Credit reporting,,,,CAPITAL ONE FINANCIAL CORPORATION,MD,21218,Older American,N/A,Web,12/02/2011,Closed with relief,Yes,Yes,2261
12/01/2011,Credit card,,Credit repo

In [31]:
cc.printSchema()

root
 |-- date_received: string (nullable = true)
 |-- product: string (nullable = true)
 |-- sub-product: string (nullable = true)
 |-- issue: string (nullable = true)
 |-- sub-issue: string (nullable = true)
 |-- consumer_complaint_narrative: string (nullable = true)
 |-- company_public_response: string (nullable = true)
 |-- company: string (nullable = true)
 |-- state: string (nullable = true)
 |-- zip_code: string (nullable = true)
 |-- tags: string (nullable = true)
 |-- consumer_consent_provided: string (nullable = true)
 |-- submitted_via: string (nullable = true)
 |-- date_sent_to_company: string (nullable = true)
 |-- company_response_to_consumer: string (nullable = true)
 |-- timely_response: string (nullable = true)
 |-- consumer_disputed: string (nullable = true)
 |-- complaint_id: string (nullable = true)



Change data type of the date_received column to timestamp and create year column
Now, let's change the data type of the date_received from string to timestamp format and create year column from it. We will use the year column for partitioning.

In [34]:
from datetime import datetime
from pyspark.sql.functions import col, udf, year

to_date_time =  udf(lambda x: datetime.strptime(x, '%m/%d/%Y'), TimestampType())
cc_2 = cc.withColumn('date_received', to_date_time(col('date_received'))).withColumn('year', year('date_received'))
cc_2.show(10)

+-------------------+--------------------+--------------------+--------------------+--------------------+----------------------------+-----------------------+--------------------+-----+--------+-------------+-------------------------+-------------+--------------------+----------------------------+---------------+-----------------+------------+----+
|      date_received|             product|         sub-product|               issue|           sub-issue|consumer_complaint_narrative|company_public_response|             company|state|zip_code|         tags|consumer_consent_provided|submitted_via|date_sent_to_company|company_response_to_consumer|timely_response|consumer_disputed|complaint_id|year|
+-------------------+--------------------+--------------------+--------------------+--------------------+----------------------------+-----------------------+--------------------+-----+--------+-------------+-------------------------+-------------+--------------------+----------------------------+

**Create Partitioned Table**

In [0]:
cc.sample(False, 0.5, 42).createOrReplaceTempView('cc_table')
cc_2.sample(False, 0.5, 42).createOrReplaceTempView('cc2_table')

In [37]:
spark.sql("select * from cc_table").show(10)

+-------------+--------------------+--------------------+--------------------+--------------------+----------------------------+-----------------------+--------------------+-----+--------+-------------+-------------------------+-------------+--------------------+----------------------------+---------------+-----------------+------------+
|date_received|             product|         sub-product|               issue|           sub-issue|consumer_complaint_narrative|company_public_response|             company|state|zip_code|         tags|consumer_consent_provided|submitted_via|date_sent_to_company|company_response_to_consumer|timely_response|consumer_disputed|complaint_id|
+-------------+--------------------+--------------------+--------------------+--------------------+----------------------------+-----------------------+--------------------+-----+--------+-------------+-------------------------+-------------+--------------------+----------------------------+---------------+------------

In [38]:
spark.sql("select * from cc2_table").show(10)

+-------------------+--------------------+--------------------+--------------------+--------------------+----------------------------+-----------------------+--------------------+-----+--------+-------------+-------------------------+-------------+--------------------+----------------------------+---------------+-----------------+------------+----+
|      date_received|             product|         sub-product|               issue|           sub-issue|consumer_complaint_narrative|company_public_response|             company|state|zip_code|         tags|consumer_consent_provided|submitted_via|date_sent_to_company|company_response_to_consumer|timely_response|consumer_disputed|complaint_id|year|
+-------------------+--------------------+--------------------+--------------------+--------------------+----------------------------+-----------------------+--------------------+-----+--------+-------------+-------------------------+-------------+--------------------+----------------------------+

In [39]:
spark.sql("select YEAR(date_received) from cc_table").show(10)

+---------------------------------+
|year(CAST(date_received AS DATE))|
+---------------------------------+
|                             null|
|                             null|
|                             null|
|                             null|
|                             null|
|                             null|
|                             null|
|                             null|
|                             null|
|                             null|
+---------------------------------+
only showing top 10 rows

